In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import cv2

import skimage.measure

import rasterio
from rasterio.features import shapes

import matplotlib.patches as mpatches
from shapely.geometry import Point, Polygon, shape, mapping
import shapely
import geopandas as gpd

from matplotlib.path import Path
import laspy
import open3d as o3d

from skopt import BayesSearchCV

import glob
import os


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Find paths to images

In [2]:
def GenPath(path):
    if path[-1] == '/':
        return path
    else:
        return path+'/'

def GetPathRelations(path):
    full_path_to_data = GenPath(path)
    
    ground_removed_image_paths = []
    laz_point_cloud_paths = []
    
    # Find full path to all images
    for path in glob.glob(full_path_to_data+'ImagesGroundRemoved/*'):
        ground_removed_image_paths.append(path)
 
    # Find full path to all laz files
    for path in glob.glob(full_path_to_data+'LazFilesWithHeightParam/*'):
        laz_point_cloud_paths.append(path)

    # Create a relation between indexes of the paths
    relation_dict = {}
    for i in range(len(ground_removed_image_paths)):
        getName = os.path.basename(ground_removed_image_paths[i])[:-8]
        relation_dict[getName] = [None, None]
        
    for i in range(len(ground_removed_image_paths)):
        tmp_path_tif = os.path.basename(ground_removed_image_paths[i])[:-8]
        tmp_path_laz = os.path.basename(laz_point_cloud_paths[i])[:-11]
        relation_dict[tmp_path_tif][0] = i
        relation_dict[tmp_path_laz][1] = i
        
    return ground_removed_image_paths, laz_point_cloud_paths, relation_dict

path_to_folders = '/home/frederik/data/TestData/data/'
ground_removed_image_paths, laz_point_cloud_paths, relation_dict = GetPathRelations(path_to_folders)

In [3]:
# VERSION 2

def GenPath(path):
    if path[-1] == '/':
        return path
    else:
        return path+'/'

def GetPathRelations(path):
    full_path_to_data = GenPath(path)
    
    ground_removed_image_paths = []
    laz_point_cloud_paths = []
    
    # Find full path to all images
    for path in glob.glob(full_path_to_data+'ImagesGroundRemoved/*'):
        ground_removed_image_paths.append(path)
 
    # Find full path to all laz files
    for path in glob.glob(full_path_to_data+'LazFilesWithHeightParam/*'):
        laz_point_cloud_paths.append(path)
        
    ground_removed_image_paths.sort()
    laz_point_cloud_paths.sort()
        
    return ground_removed_image_paths, laz_point_cloud_paths

path_to_folders = '/home/frederik/data/TestData/data/'
ground_removed_image_paths, laz_point_cloud_paths = GetPathRelations(path_to_folders)

# Create model to predict

In [64]:
class PreProcessing():

    def __init__(self):
        self
        
    def fit(self, X, y):
        return self
    
    def score(self, X,y):
        return self

In [65]:
def score(pct_lost_powerline, pct_lost_datapoints):
    s = 0
    if pct_lost_powerline > 0.001:
        return 0
    else:
        return pct_lost_datapoints


In [4]:
ground_removed_image_paths, laz_point_cloud_paths

(['/home/frederik/data/TestData/data/ImagesGroundRemoved/PUNKTSKY_00005_1km_6134_518_max.tif',
  '/home/frederik/data/TestData/data/ImagesGroundRemoved/PUNKTSKY_00005_1km_6146_468_max.tif',
  '/home/frederik/data/TestData/data/ImagesGroundRemoved/PUNKTSKY_00005_1km_6161_465_max.tif',
  '/home/frederik/data/TestData/data/ImagesGroundRemoved/PUNKTSKY_00005_1km_6162_472_max.tif',
  '/home/frederik/data/TestData/data/ImagesGroundRemoved/PUNKTSKY_00005_1km_6163_472_max.tif',
  '/home/frederik/data/TestData/data/ImagesGroundRemoved/PUNKTSKY_00005_1km_6167_473_max.tif',
  '/home/frederik/data/TestData/data/ImagesGroundRemoved/PUNKTSKY_00005_1km_6167_474_max.tif',
  '/home/frederik/data/TestData/data/ImagesGroundRemoved/PUNKTSKY_00005_1km_6167_475_max.tif',
  '/home/frederik/data/TestData/data/ImagesGroundRemoved/PUNKTSKY_00005_1km_6168_458_max.tif',
  '/home/frederik/data/TestData/data/ImagesGroundRemoved/PUNKTSKY_00005_1km_6204_505_max.tif',
  '/home/frederik/data/TestData/data/ImagesGroundR

['/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6167_475_hag_nn.laz',
 '/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6211_474_hag_nn.laz',
 '/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6220_495_hag_nn.laz',
 '/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6204_505_hag_nn.laz',
 '/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6205_513_hag_nn.laz',
 '/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6162_472_hag_nn.laz',
 '/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6167_473_hag_nn.laz',
 '/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6161_465_hag_nn.laz',
 '/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6221_452_hag_nn.laz',
 '/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6163_472_hag

{'PUNKTSKY_00005_1km_6167_474': [0, 12],
 'PUNKTSKY_00005_1km_6167_473': [1, 6],
 'PUNKTSKY_00005_1km_6168_458': [2, 14],
 'PUNKTSKY_00005_1km_6219_494': [3, 10],
 'PUNKTSKY_00005_1km_6167_475': [4, 0],
 'PUNKTSKY_00005_1km_6134_518': [5, 11],
 'PUNKTSKY_00005_1km_6162_472': [6, 5],
 'PUNKTSKY_00005_1km_6161_465': [7, 7],
 'PUNKTSKY_00005_1km_6205_513': [8, 4],
 'PUNKTSKY_00005_1km_6211_474': [9, 1],
 'PUNKTSKY_00005_1km_6220_495': [10, 2],
 'PUNKTSKY_00005_1km_6163_472': [11, 9],
 'PUNKTSKY_00005_1km_6204_505': [12, 3],
 'PUNKTSKY_00005_1km_6146_468': [13, 13],
 'PUNKTSKY_00005_1km_6221_452': [14, 8]}

In [8]:
laz_point_cloud_paths

None


TypeError: 'NoneType' object is not iterable

['/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6134_518_hag_nn.laz',
 '/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6146_468_hag_nn.laz',
 '/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6161_465_hag_nn.laz',
 '/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6162_472_hag_nn.laz',
 '/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6163_472_hag_nn.laz',
 '/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6167_473_hag_nn.laz',
 '/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6167_474_hag_nn.laz',
 '/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6167_475_hag_nn.laz',
 '/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6168_458_hag_nn.laz',
 '/home/frederik/data/TestData/data/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6204_505_hag

In [ ]:
class CustomCrossValidation:
    
    def init():
        
    
    def split(self, X, y):
        return X, []
        
        